# Talktorial T09
# Molecular Dynamics simulation of SARS Cov2 main protease in complex with inhibitor

#### CADD Seminar 2020, AG Volkamer, Charité/FU Berlin 

#### Pietro Gerletti

Berlin, June 2020

# ---------------------------------------- DISCLAIMER ----------------------------------------

The notebook presented here is thought to be run on Google Colab but it should be possible to run it on a normal laptop, which is advised if and only if you decided that your processor is not worthy of serving you and should commit seppuku (yes, I tried and decided having the processor run at 99 °C for 28h straight was kind of sadistic).

# Learning goals


## Theory


* SARS-CoV-2 main protease and relevance for drug discovery
* Molecular dynamics
* Force fields

## Practical

* Step 0 - set up for Google Colab
* Step 1 - Load PDB 
* Step 2 - Fix the PDB file (missing atoms/residues/hydrogens)
* Step 3 - Merge Molecule and Ligand 
* Step 4 - Set up the MD simulation
* Step 5 - Run the simulation

# References

* De Vivo M, Masetti M, Bottegoni G, Cavalli A. Role of Molecular Dynamics and Related Methods in Drug Discovery. J Med Chem. 2016;59(9):4035‐4061. doi:10.1021/acs.jmedchem.5b01684

* Sereina Riniker, Fixed-Charge Atomistic Force Fields for Molecular Dynamics Simulations in the Condensed Phase: An Overview. Journal of Chemical Information and Modeling 2018 58 (3), 565-578, DOI:10.1021/acs.jcim.8b00042 

* Mesecar A.D., A taxonomically-driven approach to development of potent, broad-spectrum inhibitors of coronavirus main protease including SARS-CoV-2 (COVID-19)

* Marina Macchiagodena, Marco Pagliai, Piero Procacci, Identification of potential binders of the main protease 3CLpro of the COVID-19 via structure-based ligand design and molecular modeling,Chemical Physics Letters, Volume 750,2020, https://doi.org/10.1016/j.cplett.2020.137489.

* P. S. de Laplace. Oeuvres Complètes de Laplace. Théorie Analytique des Probabilités, volume VII Gauthier-Villars, Paris, France, 3 edition, 1820.

* Schlick T. (1996) Pursuing Laplace’s Vision on Modern Computers. In: Mesirov J.P., Schulten K., Sumners D.W. (eds) Mathematical Approaches to Biomolecular Structure and Dynamics. The IMA Volumes in Mathematics and its Applications, vol 82. Springer, New York, NY




* https://github.com/openmm/openmmforcefields
* https://github.com/jaimergp/uab-msc-bioinf/blob/master/MD%20Simulation%20and%20Analysis%20in%20a%20Notebook.ipynb
* http://htmlpreview.github.io/?https://raw.github.com/pandegroup/pdbfixer/master/Manual.html
* https://en.wikipedia.org/wiki/AMBER
* https://en.wikipedia.org/wiki/Force_field_(chemistry)

# Step 0 - Google Colab set up 

In [3]:
import sys
# Download miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
# install required modules
!conda install -q -y -c omnia --prefix /usr/local/ python=3.6 openmm;
!conda install -q -y -c omnia -c conda-forge --prefix /usr/local/ python=3.6 openmmforcefields;
!conda install -q -y -c omnia -c conda-forge --prefix /usr/local/ python=3.6 rdkit;
!conda install -q -y -c omnia -c conda-forge --prefix /usr/local/ python=3.6 pdbfixer;
#!conda install -q -y -c conda-forge --prefix /usr/local/ python=3.6 mdanalysis;
!conda install -q -y -c omnia -c conda-forge --prefix /usr/local/ python=3.6 mdtraj;
# Append path to enable running packages installed with conda
sys.path.append('/usr/local/lib/python3.6/site-packages')

--2020-10-26 10:47:00--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93052469 (89M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  88.74M  68.4MB/s    in 1.3s    

2020-10-26 10:47:01 (68.4 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [93052469/93052469]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - ca-certificates==2020.1.1=0
    - certifi==2020.4.5.1=py38_0
    - cffi==1.14.0=py38he30daa8_1
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.1=py38h7b6447c_0
    - conda==4.8.3=py38_0
    - 

# Theory

The main focus of this notebook will be on how to perform molecular dynamics (MD) simulations of protein, using the main protease of SARS-CoV-2 in complex with the X77 inhibitor. Nevertheless a short introduction to the protein and its relevance in drug discovery are give below, before focusing on MD and force fields. 

## Main protease of SARS-CoV-2 

SARS-CoV-2 genome encodes, between others, polyproteins that are cleaved and tansformed in non-structural proteins by proteases. These non-structural proteins play a fundamental role in the pathogenesis and blocking their production could significally hinder the virulence of SARS-CoV-2. A possible way of doing do is to inhibit the viral proteases needed for the correct folding of the non-structural proteins. This makes the main protease of SARS-CoV-2 a candidate target for the development of a much needed antiviral drug. Here we used the pdb structure of the protease in complex with the inhibitor X77 (shown below) provided by Mesecar (2020) to perform a MD simulation, providing a 1 ns molecular trajactory. 

![SARS-CoV-2](https://cdn.rcsb.org/images/rutgers/w6/6w63/6w63.pdb1-500.jpg)

## The X77 ligand

The molecular structure of the ligand X77, which shows inhibiting properties when in complex with the SARS-CoV-2 main protease, is shown below (fetched from SMILES).

In [ ]:
from rdkit import Chem
# get the X77 structure from SMILES and draw it
X77 = Chem.MolFromSmiles(
    'CC(C)(C)c1ccc(cc1)N([C@@H](C(=O)NC2CCCCC2)c3cccnc3)C(=O)c4c[nH]cn4')
X77

## Molecular dynamics

MD is a computational method for analyzing how the atoms and molecules of a system move and interact with each other. The method stems from theoretical physics, where it was developed in the years around 1950, although the ideas behind it can be dated much earlier:

#### "An intelligence which could, at  any moment, comprehend all the forces by  which nature is animated and the  respective positions of the  beings of which it is  composed, and moreover, if this intelligence were far-reaching enough to subject these data to  analysis, it would encompass in that formula both the movements of the  largest bodies in  the universe and those of the lightest atom: to it nothing would be uncertain, and the  future, as well as the past, would be present to its eyes. The human mind offers us, in the perfection which it has  given to  astronomy, a faint sketch of this intelligence."

#### - Pierre Simon de Laplace, 1820 A.D.

Well, without entering the much-off-topic discussion about wheter it is possible to actually predict every future state of a system by knowing EVERYTHING about its components and the laws governing it (which would take us through discussing chaos and randomness in mathematical phylosophy up to relativity vs quantum physics and beyond... but if you are interested in the topic(s), here is a very interestig documentary https://www.youtube.com/watch?v=XDpurdHKpb8), let us just take this statement by de Laplace as the ideological substrate underneath molecular dynamics simulations. In other terms, we can approximate the behaviour of a physical system by knowing the characteristics of its components and applying Newton's motion laws. By solving the equations of motion, we can obtain a molecular trajectory of the system, i.e. a series of snapshots with the positions and velocities of all its particles, as well as its potential energy. To do so, we define functions, called force fields, which provide an approximate description of all the forces applied to each particle in the system. We then can use numerical integrators to solve the initial value problem for the system and obtain the trajectory. As it sounds, the process requires quite a bit of processing power and it was only few years ago that MD started seeing a more widespread use, especially in the field of computational chemistry and biology, as well as in drug discovery. To get a better idea about how the process works, here it is a GIF showing a MD simulation of a molecular motor.

<p><a href="https://commons.wikimedia.org/wiki/File:MD_rotor_250K_1ns.gif#/media/File:MD_rotor_250K_1ns.gif"><img src="https://upload.wikimedia.org/wikipedia/commons/b/b6/MD_rotor_250K_1ns.gif" alt="MD rotor 250K 1ns.gif"></a><br>By Palma, C.-A.; Kühne, D.; Klappenberger, F.; Barth, J.V. - Technische Universität München, Public Domain, <a href="https://commons.wikimedia.org/w/index.php?curid=34866205">Link</a></p>

We will now dig a bit deeper in the equations behind MD, the force fields, to gain a better undestanding how what we will be doing in the practical part.

## Force fields



Since I started the previous section with a fancy citation, it seemed fair to me to keep it up:

  ### "The force is strong with this one"

 <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ8cX8WaivCV9VcoqkuJ34KfUuRadnpP2nZGw&usqp=CAU" width="200" height="200">

  #### - Darth Vader talking about X77

And that's about it. Just ask Vader.

Force jokes a(dark)side, force fields describe the forces between the atoms whithin the molecules and between the molecules themselves. They are parametric equations with different components, each corresponding to a precise interaction (e.g. bond torsion angles, steric hindrance, etc.). The parameters values are usually derived experimentally and change for each MD scenario, depending on the molecules involved and the simulations settings. The result is a mathematical description of the energy landcape of the system, in which the forces acting on each particle result from the gradient of the potential energy with respect to the coordinates of the atoms.

A lot of different force fields are available, each with its own charachteristics. In this notebook, we will use the AMBER force fields, which are widely used for MD simulations of proteins. Their functional form is:

 <img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/fdc7cf40fe4d2f568921977c83e49f699d780e92" width="500" height="500">
 <img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/2d244cd7f2e174af73dfc1e4dd0b37083115b690" width="500" height="200">

From the formula you can see there are different components, each describing a different interaction: the first two contain information about bonds lenght and their angles (intramolecular forces), while the third component describes intermolecular forces as the electrostatic interactions. Note that these force fields assume fixed-charge particles and do not allow polarization, nor they consider how a local charge influences its surroundings. It follows a visual representation of force fields components, which shows the same concepts in a more intuitive way.

<p><a href="https://commons.wikimedia.org/wiki/File:MM_PEF.png#/media/File:MM_PEF.png"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5c/MM_PEF.png/1200px-MM_PEF.png" width=400 height=400  alt="MM PEF.png"></a><br>By <a href="//commons.wikimedia.org/w/index.php?title=User:Edboas&amp;action=edit&amp;redlink=1" class="new" title="User:Edboas (page does not exist)">Edboas</a> - <span class="int-own-work" lang="en">Own work</span>, <a href="https://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=4194424">Link</a></p>

After selecting a force field to run the MD with, the equation needs to be parametrized according to the specific scenario we wish to simulate. In this process, an algorithm will fetch the parameters needed (in this case we rely on `openforcefields`), taking into account the simulation settings. Usually the process involves these basic steps (but it can vary):

* fetch the crystal structure/chemical formula
* define atom types
* get atomic charges
* assign initial Lennard-Jones and bond parameters 
* test density and geometry (reference data)
* test energetic properties (surface and hydration energy - compare with reference data)

Once the force field is parametrized, the simulation basically consists in solving a differential equations system and saving the resulting trajectory.

Wrapping up, molecular dynamics simulate the trajectory of a physico-chemical system by using force fields to describe the interactions between each atom in the system, applying classical physics' law of motions and solving the equations by employing numerical methods that take advantage of the high performance processors of modern computers. This means no statistical mechanics nor quantum physics knowledge is included in the method, although there is ongoing reserach in the field of quantum force fields. Surprisingly, such deterministic, classical-physics-based description of a physico-chemical system works well enough to provide us with useful insights into what happens at a spatiotemporally not observable scale.

# Practical

We will now proceed to implement a simple MD simulation using `openmm` and `openforcefields` on Google Colab. There are four funtamental steps needed to set up and run the simulation, plus an extra pre-step (step 0) needed to set up the notebook on Google Colab (done at the beginning).

## Step 1 - PDB file upload

After setting up Google Colab, we need a file to work with. In this case we will run the simulation on the SARS Cov2 main protease in complex with an inhibitor. We need therefore to import all needed packages and to upload the file(s) we need to work with.

In [3]:
# imports
import io

import mdtraj as md
import numpy as np
import parmed
import simtk.openmm as mm
import simtk.openmm.app as app
from simtk.openmm import LangevinIntegrator
from simtk.openmm import unit
from simtk.openmm.app import CheckpointReporter, ForceField, PDBFile
from openforcefield.topology import Molecule, Topology
from openmmforcefields.generators import GAFFTemplateGenerator


import pdbfixer
from rdkit import Chem
from rdkit.Chem import AllChem
import copy

In [ ]:
from google.colab import files

In [6]:
#in Google Colab
uploaded = files.upload()

Saving 6w63.pdb to 6w63.pdb


In [ ]:
path = "6w63.pdb"

In [7]:
#in Jupyter Notebook
path = "data/6w63.pdb"

## Step 2 - fix PDB file for missing atoms and residues

In case our pdb file is missing some hydrogen atoms or some residues we need to add them and thus "fix" the structure. To do this there are several options, a common one in python being the package `pdbfixer`.


In [8]:
fixer = pdbfixer.PDBFixer(path) #fix pdb
fixer.removeHeterogens()
fixer.findMissingResidues() # find missing residues
fixer.missingResidues = {}
fixer.findNonstandardResidues() # find non standard residue#
fixer.replaceNonstandardResidues()# replace non standard residues with standard one
fixer.findMissingAtoms() # find missing heavy atoms
fixer.addMissingAtoms() # add missing atoms and residues
app.PDBFile.writeFile(fixer.topology, fixer.positions, open('6w63_fixed.pdb', 'w'))

To run a reliable simulation it is crucial to have correctly protonated protein and ligand structures, but the H-Atoms are often not included in topology files and have to be added before simulating. Most libraries handle proteins rather well, but fixing small molecules like ligands is a different task.
Thus we protonate our ligand with RDKit, which specialises on small molecules.
We can extract the ligand from the complex by splitting it into residues and selecting the ligand.

In [9]:
mol = Chem.MolFromPDBFile(path)
lig_resname = 'X77'
mol_split = Chem.rdmolops.SplitMolByPDBResidues(mol)
lig_mol_wo_bond_orders = copy.deepcopy(mol_split[lig_resname])
lig_mol_wo_bond_orders_wo_Hs = Chem.RemoveHs(lig_mol_wo_bond_orders)

In PDB Topology files no bond order information is given. RDKit needs the information to correctly add Hydrogens, thus we need to assign bond orders to our ligand. 
We take a molecule generated from a SMILES string as a reference for adding bond order information.
Finally, RDKit can add the Hydrogens.

In [10]:
reference_mol = Chem.MolFromSmiles('CC(C)(C)c1ccc(cc1)N([C@@H](C(=O)NC2CCCCC2)c3cccnc3)C(=O)c4c[nH]cn4')
lig_mol = AllChem.AssignBondOrdersFromTemplate(reference_mol, lig_mol_wo_bond_orders_wo_Hs)
lig_mol.AddConformer(lig_mol_wo_bond_orders.GetConformer(0))
lig_mol_H = Chem.rdmolops.AddHs(lig_mol, addCoords=True)
Chem.MolToMolFile(lig_mol_H, 'X77.sdf')

## Step 3 - Merge Molecule and Ligand
In the next step we want to merge the protonated ligand with the protein using MDTraj. MDTraj works from an OpenMM Topology, so we will need to do some conversions. 
First, create an OpenFF Molecule from the RDKit molecule.

In [11]:
ligand = Molecule.from_rdkit(lig_mol_H)

Before we continue, lets add naming information back that we lost with RDKit in the previous step. Names are not necessary for the simulation, but will help with keeping generated files human-readable and make working with the simulation easier later on.

In [12]:
ligand.name = "X77"

In [13]:
atom_names = {
        1: "H",
        6: "C",
        7: "N",
        8: "O"
    }
for atom in ligand.atoms:
  atom.name = atom_names.get(atom.atomic_number)

Now we convert the OpenFF Topology to an OpenMM Topology. Together with the positions we have a working OpenMM system for the ligand.

In [14]:
off_ligand_topology = ligand.to_topology()
ligand_topology = off_ligand_topology.to_openmm()
ligand_positions = ligand.conformers[0]

Our Protein is already in OpenMM format.

In [15]:
# Read in the PDB and create an OpenMM topology
protein_topology, protein_positions = fixer.topology, fixer.positions

To combine the ligand and the protein structures both topologies are read into MDTraj and then joined.

In [16]:
# Add ligand to topology - credit to @hannahbrucemacdonald for help here
print("--> Combining protein and ligand topologies")
md_protein_topology = md.Topology.from_openmm(protein_topology)  # using mdtraj for protein top
md_ligand_topology = md.Topology.from_openmm(ligand_topology)  # using mdtraj for ligand top
md_complex_topology = md_protein_topology.join(md_ligand_topology)  # add them together

--> Combining protein and ligand topologies


The joined complex topology is then converted back to OpenMM. 

In [17]:
complex_topology = md_complex_topology.to_openmm()  # now back to openmm

For a complete complex we also need to add the ligand positions back into the system.
Again, the protein positions are already in the right format. The ligand positions have to be converted from  Ångström to Nanometers.

In [18]:
total_atoms = len(protein_positions) + len(ligand_positions)
#generate a quantitiy array of right size
complex_positions = unit.Quantity(np.zeros([total_atoms, 3]), unit=unit.nanometers)
complex_positions[0 : len(protein_positions)] = protein_positions #add protein positions
#Fill with ligand positions
for i, atom in enumerate(ligand_positions, len(protein_positions)):
    coords = atom / atom.unit
    complex_positions[i] = (
        coords / 10.0
    ) * unit.nanometers  # since openmm works in nm

In [19]:
app.PDBFile.writeFile(complex_topology,
                      complex_positions,
                      open('complex.pdb', 'w'))

## Step 4 - Molecular dynamics simulation set up

We can then use the fixed pdb to set up the MD simulation. We first need to fetch the ligand molecule from its SMILES and then we can use GAFF to generate a template for it, which is needed for the force field to be set up properly (otherwise the force field has "no knowledge" of the ligand). We can then create the MD environment containing the protein and the ligand and add the solvent we wish to simulate in, in this case water. Once the system is ready we save its pdb file and proceed in setting up the system.

In [28]:
# Create an openforcefield Molecule object for ligand X77 from SMILES
off_ligand = Molecule.from_smiles(
    'CC(C)(C)c1ccc(cc1)N([C@@H](C(=O)NC2CCCCC2)c3cccnc3)C(=O)c4c[nH]cn4')
# Create the GAFF template generator
gaff = GAFFTemplateGenerator(molecules=off_ligand)
# Create an OpenMM ForceField object with AMBER ff14SB
# and TIP3P with compatible ions
forcefield = app.ForceField(
    '/usr/local/lib/python3.6/site-packages/simtk/openmm/app/data/amber14/protein.ff14SB.xml',
    '/usr/local/lib/python3.6/site-packages/simtk/openmm/app/data/amber14/tip3p.xml')
# Register the GAFF template generator
forcefield.registerTemplateGenerator(gaff.generator)

In [30]:
# Add solvent
modeller = app.Modeller(complex_topology, complex_positions)
modeller.addHydrogens(forcefield);
modeller.addSolvent(forcefield, padding=1.0*unit.nanometers,
                    model='tip3p', ionicStrength=0.15*unit.molar)
# Save PDB file of solvated molecule
app.PDBFile.writeFile(modeller.topology,
                      modeller.positions,
                      open('MD_system.pdb', 'w'))
# Simulation settings
system = forcefield.createSystem(modeller.topology)
integrator = LangevinIntegrator(300*unit.kelvin,
                                1.0/unit.picoseconds,
                                2.0*unit.femtoseconds)
simulation = app.Simulation(modeller.topology, system, integrator)

Alternatively we can also write this as a function returning the simulation object:

In [ ]:
def SimSet(smiles, protein):
    """ Create simulation settingd for MD simulation of protein bound to ligand

        Parameters
        ----------
        smiles : str
            the SMILES of the ligand
        protein : openmm PDBFile object
            a pdb file loaded with simtk.openmm.app.PDBfile

        Returns
        -------
        Saves the solvated system as pdb in the working directoty

        Returns simulation object (simtk.openmm.app.Simulation()) for the MD simulation.
        It includes info about the simulation settings and the parametrized ligand.

    """
    # Create an openforcefield Molecule object for ligand from SMILES
    mol = Molecule.from_smiles(smiles)
    # Create the GAFF template generator
    gaff = GAFFTemplateGenerator(molecules=mol)
    # Create an OpenMM ForceField object with AMBER ff14SB and TIP3P with compatible ions
    forcefield = app.ForceField('/usr/local/lib/python3.6/site-packages/simtk/openmm/app/data/amber14/protein.ff14SB.xml',
                                '/usr/local/lib/python3.6/site-packages/simtk/openmm/app/data/amber14/tip3p.xml')
    # Register the GAFF template generator
    forcefield.registerTemplateGenerator(gaff.generator)
    # Add solvent
    modeller = app.Modeller(protein.topology, protein.positions)
    modeller.addSolvent(forcefield, padding=1.0*unit.nanometers, model='tip3p', ionicStrength=0.15*unit.molar)
    # Save PDB file of solvated molecule
    app.PDBFile.writeFile(modeller.topology, modeller.positions, open('MD_system.pdb', 'w'))
    # Simulation settings
    system = forcefield.createSystem(modeller.topology)
    integrator = LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
    simulation = app.Simulation(modeller.topology, system, integrator)
    return simulation

## Step 5 - Run the MD simulation
Now that everything is set-up we can run the simulation. We need to set starting positions and to minimize the energy of the system. Once that is done, we can let the simulation run! Here we are simulating 1 ns and saving one molecular "snapshot" every 5000, for a total of 100 frames out of the 500000 steps we are taking. The results are saved in a .dcd files, which contains the coordinates of all the atoms at a given time point. Together with the pdb file of the system, it gives us all the information about our simulation.  

In [31]:
# Initialize the MD simulation
print('Setting up positions...')
simulation.context.setPositions(modeller.positions)

state = simulation.context.getState(getEnergy=True)
print(state.getPotentialEnergy())

Setting up positions...
-762242.375 kJ/mol


In [32]:
# Minimize energy
print('Energy minimization in progress...')
simulation.minimizeEnergy()

Energy minimization in progress...


In [34]:
# Output settings
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 5000))

# Output data
simulation.reporters.append(app.StateDataReporter(sys.stdout, 5000, step=True,
                                                  potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
                                                  speed=True, totalSteps=5*1e5, separator='\t'))

# Set velocities
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)



In [ ]:
# Run the simulation
simulation.step(5*1e5)

#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
1.0%	5000	-860003.625	300.3846344151758	0	--


In [23]:
# Download the results
files.download('MD_system.pdb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
files.download('trajectory.dcd')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We have succefully ran a MD simulation! If you want to know how to visualize the results, you can refer to the next notebook in the repository.   
   
As last I would like to add a quick note about the run-time. We saw that running this kind of simulations can be quite computationally expensive and long. Here on Google Colaboratory we achieved a speed of 3.7 ns/day, which is OK but still slow, especially if we need to run longer simulations and multiple times with different settings. I also tried running the same simulation on a Ubuntu VM on Virtualbox (Windows 10 host), achieving 1 ns/day as speed. Given that it was running on 4 virtual cores (host processor Intel core i7 8th gen) and not on the host machine 8 cores, it was probably a bit slower than it could have been, but it is clear that CPU is not the way to go for such applications. GPU parallel computations allow to reduce the run time exponentially and should always be preferred. Anyhow if, as stated at the beginning, you want to punish your laptop, you can definetly use MD simulations as an effective tool.  